In [ ]:
import pandas as pd
import cobra

from micom import Community
from micom.workflows import build, grow, tradeoff, fix_medium,build_database
from micom import load_pickle
from micom.viz import plot_tradeoff, plot_exchanges_per_sample, plot_growth

In [ ]:
Tax= pd.DataFrame(columns=['id','genus','species','reactions','metabolites','sample_id','abundance'])
Tax

In [ ]:
Tax.loc[len(Tax.index)] = ['Rhodosporidium', 'Rhodosporidium', 'R. toruloides',2398,2051,'One',500]
Tax.loc[len(Tax.index)] = ['Synechococcus', 'Synechococcus','S. elongatus',851,769,'One',500]
Tax.loc[len(Tax.index)] = ['Azotobacter','Azotobacter','A. vinelandii',2469,2003,'One',500]
Tax

## Build Database File

If necessary, construct a database file from csv 

In [ ]:
db = pd.read_csv('./man_tertiary.csv')
db

In [ ]:
build_database(db,'./db_tertiary')

In [ ]:
db_path = './db_tertiary/'

## Construct Community Model

In [ ]:
# Build Manifest object from Taxonomy DataFrame and the corresponding database directory

# Skip this step if manifest has already been built and saved to "models" directory

'''
Important: Declare the Solver you would like to use for this Community model here:
    - osqp (good for smaller models)
    - gurobi
    - glpk
    - cplex
    - scipy
'''

manifest = build(Tax, out_folder="models_tertiary", model_db=db_path, cutoff=0.0001, threads=10,solver='gurobi')
manifest

In [ ]:
# Load Model pickle file for the community object

com = load_pickle("models_tertiary/One.pickle")
print(len(com.reactions))

In [ ]:
com.medium

In [ ]:
medium_bkp = com.medium

## Optimize Model

In [ ]:
result = com.optimize(fluxes=True, pfba=True)
result

In [ ]:
result.fluxes['SUCRt2']

In [ ]:
result.fluxes['SUCRtex']

In [ ]:
com.summary()

In [ ]:
com.reactions.EX_photon410_m

### Update Medium to Show that Azotobacter Prevents Community Growth

In [ ]:
# Restore medium to original
com.medium = medium_bkp

# Set variable to become new medium
medium_to_change = com.medium

#Add or subtract reactions

#Remove carbon sources
medium_to_change["EX_sucr_m"] = 0
medium_to_change["EX_glc__D_m"] = 0
medium_to_change["EX_leu__L_m"] = 0
medium_to_change["EX_hco3_m"] = 0
#medium_to_change["EX_glyc__R_m"] = 0
medium_to_change["EX_nh4_m"] = 0
medium_to_change["EX_no3_m"] = 0
medium_to_change["EX_n2_m"] = 1.76

#Remove unnecessary photons
medium_to_change["EX_photon410_m"] = 0
medium_to_change["EX_photon430_m"] = 0
medium_to_change["EX_photon450_m"] = 0
medium_to_change["EX_photon470_m"] = 0
medium_to_change["EX_photon490_m"] = 0
medium_to_change["EX_photon510_m"] = 0
medium_to_change["EX_photon550_m"] = 0
medium_to_change["EX_photon570_m"] = 0
medium_to_change["EX_photon590_m"] = 0
medium_to_change["EX_photon610_m"] = 0
medium_to_change["EX_photon630_m"] = 0
medium_to_change["EX_photon650_m"] = 0
medium_to_change["EX_photon670_m"] = 0


# Set the new medium as the model's medium
com.medium = medium_to_change
com.medium

In [ ]:
result_altered_medium = com.optimize(fluxes=True, pfba=True)

In [ ]:
result_altered_medium

In [ ]:
result_altered_medium.fluxes['SUCRtex']

In [ ]:
com.summary()

In [ ]:
result_altered_medium.fluxes.to_csv('ternary_model_co2_n2_growth_reduced_photons.csv')

In [ ]:
from IPython.display import HTML

In [ ]:
HTML(result_altered_medium.fluxes.T.to_html())

In [ ]:
result_altered_medium.fluxes.EX_photon550_m

In [ ]:
com.medium